## Combine page id and redirect to correct the links

In [1]:
import pandas as pd
import time

In [2]:
# Load the data
start_t = time.time()
redirect_df = pd.read_pickle('/home/benjamin/wikipedia/Wikipedia/redirect_parsed0.gz')
print('redirect loaded')
pageid_df = pd.read_pickle('/home/benjamin/wikipedia/Wikipedia/pageid_parsed0.gz')
print('page loaded')
pagelinks_df = pd.read_pickle('/home/benjamin/wikipedia/Wikipedia/pagelinks_parsed0.gz')
print('pagelinks loaded')
print('Load Time:', time.time() - start_t)

redirect loaded
page loaded
pagelinks loaded
Load Time: 9.46012282371521


In [3]:
pagelinks_df.head()

,pageId,pageTitle
0,9773,!
1,15154,!
2,25213,!
3,1028188,!
4,1340746,!


## Change source Ids to titles in redirect

In [4]:
print(pageid_df.columns)
pageid_df.set_index('pageId', inplace=True)

Index(['pageId', 'pageTitle'], dtype='object')


In [5]:
print('Old name',redirect_df.columns)
redirect_df.rename(columns={'pageId' : 'initial_id', 'pageTitle' : 'fix_target'}, inplace=True)
print('New name',redirect_df.columns)

Old name Index(['pageId', 'pageTitle'], dtype='object')
New name Index(['initial_id', 'fix_target'], dtype='object')


In [6]:
redirect_df = redirect_df.join(pageid_df, on='initial_id', how='left')
redirect_df.rename(columns={'pageTitle' : 'initial_target'}, inplace=True)
redirect_df = redirect_df.drop('initial_id', axis=1)
redirect_df = redirect_df.set_index('initial_target')

Check for NaN values

In [7]:
len(redirect_df)

8500104

In [8]:
redirect_df.isnull().sum()

fix_target    0
dtype: int64

In [9]:
redirect_df[redirect_df.isnull().any(axis=1)]

,fix_target
initial_target,


In [8]:
#redirect_df = redirect_df.reindex(columns=['initial_id','initial_target','fix_target'])

In [10]:
redirect_df.head()

,fix_target
initial_target,
AccessibleComputing,Computer_accessibility
AfghanistanHistory,History_of_Afghanistan
AfghanistanGeography,Geography_of_Afghanistan
AfghanistanPeople,Demographics_of_Afghanistan
AfghanistanCommunications,Communications_in_Afghanistan


## Replacing target in pagelinks by the redirected one

In [11]:
pagelinks_df.head()

,pageId,pageTitle
0,9773,!
1,15154,!
2,25213,!
3,1028188,!
4,1340746,!


In [12]:
#pagelinks_df.set_index('sourceId', inplace=True)
len(pagelinks_df)

11003768

In [13]:
old_size = len(pagelinks_df)
pagelinks_df.rename(columns={'pageTitle' : 'target'}, inplace=True)
pagelinks_df = pagelinks_df.join(pageid_df, on='pageId', how='outer')
pagelinks_df.rename(columns={'pageTitle' : 'source'}, inplace=True)
new_size = len(pagelinks_df)
print(old_size,new_size)

11003768 24359904


In [14]:
pagelinks_df.isnull().sum()

pageId           0
target    13356136
source          86
dtype: int64

In [16]:
pagelinks_df[pagelinks_df.isnull().any(axis=1)]

,pageId,target,source
253291,58044482,0-4-0,NaN
254041,58044482,0-4-2,NaN
4402628,58044713,1877_Men\'s_Tennis_tour,NaN
4419523,58044713,1878_Men\'s_Tennis_tour,NaN
4436437,58044713,1879_Men\'s_Tennis_tour,NaN
4459036,58044713,1880_Men\'s_Tennis_tour,NaN
4494754,58044713,1881_Men\'s_Tennis_tour,NaN
4514671,58044713,1882_Men\'s_Tennis_tour,NaN
4536196,58044713,1883_Men\'s_Tennis_tour,NaN
4559070,58044713,1884_Men\'s_Tennis_tour,NaN


In [20]:
# Id list of unique pages that have an id but do not have a title
pagelinks_df['pageId'][pagelinks_df['source'].isnull()].unique()

array([58044482, 58044713])

In [15]:
pagelinks_df.head()

,pageId,target,source
0,9773,!,EBCDIC
1326,9773,"\""",EBCDIC
12977,9773,$,EBCDIC
15271,9773,%,EBCDIC
112343,9773,+,EBCDIC


In [16]:
pagelinks_df['fix_target'] = pagelinks_df['target'].copy()
pagelinks_df = pagelinks_df.set_index('target')

In [17]:
#redirect_df.rename(columns={'r_t_T' : 'fix_target'}, inplace=True)

In [18]:
pagelinks_df.update(redirect_df)
pagelinks_df.reset_index(inplace = True)
pagelinks_df = pagelinks_df.drop('pageId', axis=1)
pagelinks_df['is_redirect'] = (pagelinks_df['target'] != pagelinks_df['fix_target'])
pagelinks_df = pagelinks_df.reindex(columns=['source','target','fix_target','is_redirect'])
print(len(pagelinks_df))

11003682


In [19]:
pagelinks_df.head()

,source,target,fix_target,is_redirect
0,EBCDIC,!,Exclamation_mark,True
1,EBCDIC,"\""",Quotation_mark,True
2,EBCDIC,$,Dollar_sign,True
3,EBCDIC,%,Percent_sign,True
4,EBCDIC,+,Plus_and_minus_signs,True


In [20]:
len(pagelinks_df)

11003682

In [22]:
pagelinks_df.describe()

,source,target,fix_target,is_redirect
count,11003682,11003682,11003682,11003682
unique,694691,257558,228792,2
top,List_of_named_minor_planets:_10000–19999,1950_United_States_Census,1950_United_States_Census,False
freq,4164,23124,23126,10549505
